In [98]:
using SwiftObjectStores
using WordEmbeddings, SoftmaxClassifier
using SwiftObjectStores
using ProgressMeter
using Utils
using Query
using WordStreams
using Distances
using StatsBase
using Training
using AdaGramCompat
using CorpusLoaders.Similarity
using WordNet

  likely near /mnt_volume/phd/prototypes/SenseSplittingWord2Vec/src/WordDistributions.jl:2
  likely near /mnt_volume/phd/prototypes/SenseSplittingWord2Vec/src/WordDistributions.jl:2
  likely near /mnt_volume/phd/prototypes/SenseSplittingWord2Vec/src/WordDistributions.jl:2

Use "Dict(a=>b for (a,b) in c)" instead.


In [99]:
const WN_PATH="data/corpora/WordNet-2.1/"
#WN_PATH3 = "/usr/share/nltk_data/corpora/wordnet/"
db = DB(WN_PATH)

WordNet.DB

In [111]:
ee = get_jld(SwiftService(), "sensemodels", "plain/tokenised_lowercase_WestburyLab.wikicorp.201004_100_nosubsample.semhuff.jld", "ee");

/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/util/ssl_.py:122: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.io/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning
/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/util/ssl_.py:122: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.io/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning


In [130]:
dtree,labels = Query.nn_tree(ee)

LoadError: LoadError: DimensionMismatch("tried to assign 100-element array to 8×1 destination")
while loading In[130], in expression starting on line 1

In [131]:
length(ee.embedding)

497485

In [89]:
function related_words(lem::AbstractString, pos::Char, db::DB, max_hyponyms = 5) 
    haskey(db.lemmas[pos], lowercase(lem)) || return String[]

    ss = synsets(db, db[lem,pos])
    related_synsets = Set(ss)
    for si in ss
        union!(related_synsets, expanded_hypernyms(db, si))
        union!(related_synsets, antonyms(db, si))

        hypos = hyponyms(db, si)

        union!(related_synsets, take(hypos, max_hyponyms))
    end
    map(words, related_synsets) |> Base.flatten |> unique
end

function related_words(word::AbstractString, db::DB, max_hyponyms = 5) 
    related_words_of_pos(pos) = related_words(word, pos, db, max_hyponyms) 
    
    WordNet.SYNSET_TYPES |> keys |> pos->map(related_words_of_pos, pos) |> Base.flatten |> unique
end



related_words (generic function with 4 methods)

128-element Array{Any,1}:
 "fire"          
 "firing"        
 "criticism"     
 "process"       
 "affliction"    
 "occurrence"    
 "occurrent"     
 "happening"     
 "oxidation"     
 "oxidisation"   
 "oxidization"   
 "trouble"       
 "entity"        
 "state"         
 "shelling"      
 "battery"       
 "bombardment"   
 "barrage"       
 "substance"     
 "matter"        
 "flare"         
 "operation"     
 "passionateness"
 "passion"       
 "element"       
 "content"       
 "message"       
 "abstraction"   
 "bonfire"       
 "balefire"      
 "artefact"      
 "artifact"      
 "communication" 
 "act"           
 "broadside"     
 "trial"         
 "visitation"    
 "tribulation"   
 "enclosure"     
 "burning"       
 "combustion"    
 "fusillade"     
 "salvo"         
 "burst"         
 "volley"        
 "backfire"      
 "activity"      
 "blaze"         
 "blazing"       
 "disapproval"   
 "event"         
 "cookfire"      
 "ignition"      
 "onrush"        
 "

In [114]:
ee|> typeof

WordEmbeddings.WordEmbedding

In [112]:
normal_probs(logprobs::Vector) = normal_probs(copy(logprobs))
function normal_probs!{F<:AbstractFloat}(logprobs::Vector{F})
    ret = logprobs
    max_lp = maximum(logprobs)
    ret.-=max_lp #Bring closer to zero
    map!(exp,ret)
    denom = sum(ret)
    ret./=denom
    ret
end

function general_wsd(ee, context, wvs, priors=ones(length(wvs)))
    lps = Vector{Float64}(length(wvs))
    for (ii,(prior, wv)) in enumerate(zip(priors, wvs))
        lps[ii] = Query.logprob_of_context(ee, context, wv; skip_oov=true, normalise_over_length=true)
        lps[ii] += log(prior)
    end
    
    normal_probs!(lps)
end

    

general_wsd (generic function with 2 methods)

In [121]:
function all_word_vectors(ee::WordEmbedding, word)
    if haskey(ee.embedding, word)
        [ee.embedding[word]]
    else
        Vector{Float32}[]
    end
end

function synthesize_embedding(ee::WordEmbedding,context::AbstractVector, wn::DB, lemma::AbstractString, pos::Char)
    words = related_words(lemma, pos, db)
    
    wvs = vcat((all_word_vectors(ee,w) for w in words)...)
    if length(wvs) == 0
        fallbacks = split(fallback_word_or_phrase, " ")
        wvs = vcat((all_word_sense_vectors(ee,w) for w in fallbacks)...)
        if length(wvs) == 0
            throw(KeyError(" $words, nor $fallbacks have embeddings"))
        end
    end

    probs = general_wsd(ee, context, wvs)
    sum(wvs.*probs)
end

synthesize_embedding (generic function with 1 method)

In [133]:
wv1 = synthesize_embedding(ee, split("he me i lost my job and now i have no money"), db, "fire", 'v')
wv2 = synthesize_embedding(ee, split("we were camping and it was cold but the kept us warm"), db, "fire", 'n')
wv3 = synthesize_embedding(ee, split("he was made and paid off"), db, "redundant", 'a')

100-element Array{Float32,1}:
 -0.171361 
 -0.0601222
 -0.130508 
 -0.129067 
 -0.0248395
 -0.0265742
 -0.0515996
 -0.0724668
 -0.0111354
 -0.104404 
 -0.105999 
 -0.19579  
 -0.10982  
 -0.137117 
 -0.0670138
 -0.164955 
  0.0997669
 -0.0129256
 -0.0563204
 -0.0667889
 -0.160745 
 -0.0585518
 -0.0935155
 -0.0839081
 -0.149451 
 -0.0220395
 -0.0955562
 -0.143061 
 -0.0422845
 -0.0328509
 -0.0767439
 -0.118051 
 -0.195949 
 -0.0917757
 -0.113751 
 -0.182714 
 -0.127367 
 -0.166535 
 -0.231383 
 -0.239143 
 -0.136593 
 -0.153564 
 -0.0879698
 -0.190549 
 -0.0565284
 -0.087515 
 -0.038529 
 -0.151196 
 -0.165513 
 -0.109016 
 -0.0509603
 -0.123052 
 -0.121415 
 -0.100983 
 -0.0942173
 -0.0814664
 -0.0411696
 -0.012768 
 -0.122486 
 -0.122311 
 -0.0406796
 -0.16835  
 -0.137209 
 -0.0511401
 -0.173059 
 -0.0728902
 -0.056107 
 -0.010812 
 -0.32882  
 -0.0953964
 -0.0745012
 -0.148656 
 -0.103872 
 -0.0951794
 -0.0695007
 -0.140382 
 -0.0685524
 -0.0745986
 -0.165295 
 -0.186402 
 -0.117548

In [134]:
Query.angular_dist(wv1, wv2)

0.12487596f0

In [135]:
Query.angular_dist(wv1, wv3)

0.17384131f0

In [136]:
Query.angular_dist(wv2, wv3)

0.20049803f0